# Diesel Random Fields Module

This module provides capabilities for creating and sampling from- random fields on a large scale. 
Its core functionality is handling very large covariance matrices through lazy Dask arrays. 

Key functionalities include:
- **creating large covariance matrices from standard kernels** (Gaussian, Matérn, ...)
- **sampling random fields at scale**

This notebook demonstrates core capabilities of this module.

In [ ]:
import xarray as xr
import dask.array as da
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from diesel.random_fields import Matern32
from diesel.gridding import unflatten_to_grid, flatten_grid

## 1. Create a synthetic dataset
In `DIESEL`, large datasets of geographical data are handled by [Xarray](https://xarray.dev/). In this example, we 
create an example gridded dataset on a regular rectangular latitude-longitude grid. For the sake of generality, we also include a time 
dimension.

In [ ]:
lat = np.linspace(-90, 90, 50)
lon = np.linspace(-180, 180, 100)
time = np.arange(10)  # 10 time steps

grid = xr.Dataset(
    {
        "latitude": ("latitude", lat),
        "longitude": ("longitude", lon),
        "time": ("time", time),
    }
)
grid

Since all data assimilation algorithms make use of **flattened** vector data, `DIESEL` provides functionalities for getting flat coordinates 
vector out of a grid. Users need to specify the coordinates along which to flatten.

In [ ]:
points = flatten_grid(grid, ["latitude", "longitude"])
points

## 2. Creating a Covariance kernel
Covariance kernel objects can be created by specifying variance $\sigma^2$ and lengthscales $\vec{\lambda}$. For Matérn kernel of degree $\nu$ the convention is 
$$
k(\vec{x}, \vec{y}) = \sigma^2 \frac{2^{1-\nu}}{\Gamma(\nu)} 
\left(
    1\nu \sum_i (\frac{x_i - y_i}{\lambda_i})^2
\right)^{\nu/1}
\mathcal{K}_{\nu}
\left(
 \left(
    1\nu \sum_i (\frac{x_i - y_i}{\lambda_i})^2
\right)^{\nu}   
\right)
$$

In [4]:
# Initialize the Matérn 3/2 kernel
matern32 = Matern32(sigma=2.0, l=[30, 30])

# Compute the covariance matrix
cov_matrix = matern32(points, points)

# Trigger computation (optional)
result = cov_matrix.compute()

In [ ]:
mean = np.zeros(result.shape[0])  # Mean vector of zeros
samples = np.random.multivariate_normal(mean, result, size=1)  # Generate one sample
samples

In [ ]:
sample_gridded = unflatten_to_grid(
    samples.reshape(-1), grid, active_dims=["latitude", "longitude"]
)
sample_gridded.plot()

In [ ]:
import cartopy.crs as ccrs


p = sample_gridded.plot(
    subplot_kws=dict(projection=ccrs.Orthographic(-80, 35), facecolor="gray"),
    transform=ccrs.PlateCarree(),
    cbar_kwargs={"label": "temperature anomaly (°C)"},
)

p.axes.set_global()

p.axes.coastlines()